In [93]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json
from datetime import datetime 
from alpaca_trade_api.rest import REST, TimeFrame
import tweepy
%matplotlib inline
from pathlib import Path
import csv
import re

In [131]:
# Load .env enviroment variables
load_dotenv()
bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
api_key =  os.getenv('TWITTER_API_KEY')
api_secret_key =  os.getenv('TWITTER_SECRET_KEY')
access_token =  os.getenv('TWITTER_ACCESS_TOKEN')
access_secret_token =  os.getenv('TWITTER_SECRET_TOKEN')


def get_api_keys():
    api_key =  os.getenv('TWITTER_API_KEY')
    api_secret_key =  os.getenv('TWITTER_SECRET_KEY')
    bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
    access_token =  os.getenv('TWITTER_ACCESS_TOKEN')
    access_secret_token =  os.getenv('TWITTER_SECRET_TOKEN')
    return api_key,api_secret_key,bearer_token,access_token,access_secret_token

def get_api_keys1():
    api_key =  os.getenv('access_token')
    api_secret_key =  os.getenv('access_token_secret')
    bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
    access_token =  os.getenv('consumer_key')
    access_secret_token =  os.getenv('consumer_secret')
    return api_key,api_secret_key,bearer_token,access_token,access_secret_token

api_key,api_secret_key,bearer_token,access_token,access_secret_token = get_api_keys()


In [130]:
url = "https://api.twitter.com/2/tweets/1535608320211427329?tweet.fields=created_at,attachments&expansions=author_id"

payload={}
headers = {
  'id': '1228393702244134912',
  'Authorization': 'Bearer '+ bearer_token,
  'Cookie': 'guest_id=v1%3A165495549908649152'
}

response = requests.request("GET", url, headers=headers, data=payload)
json_response = response.text
print(json_response)

{"data":{"text":"On this day in 1980: Steve Ballmer became Microsoft employee #24.\n\nHis shares are now worth $81 billion. https://t.co/tHc8wtldIG","author_id":"164425558","attachments":{"media_keys":["3_1535608317837443073"]},"created_at":"2022-06-11T13:02:08.000Z","id":"1535608320211427329"},"includes":{"users":[{"id":"164425558","name":"Jon Erlichman","username":"JonErlichman"}]}}


In [28]:
query = "NFT"
client = tweepy.Client(bearer_token=bearer_token)
counts = client.get_recent_tweets_count(query=query,granularity="day")
for count in counts.data:
    print(count)

{'end': '2022-06-07T00:00:00.000Z', 'start': '2022-06-06T17:21:19.000Z', 'tweet_count': 474374}
{'end': '2022-06-08T00:00:00.000Z', 'start': '2022-06-07T00:00:00.000Z', 'tweet_count': 2040590}
{'end': '2022-06-09T00:00:00.000Z', 'start': '2022-06-08T00:00:00.000Z', 'tweet_count': 2003730}
{'end': '2022-06-10T00:00:00.000Z', 'start': '2022-06-09T00:00:00.000Z', 'tweet_count': 2219720}
{'end': '2022-06-11T00:00:00.000Z', 'start': '2022-06-10T00:00:00.000Z', 'tweet_count': 2334905}
{'end': '2022-06-12T00:00:00.000Z', 'start': '2022-06-11T00:00:00.000Z', 'tweet_count': 1968336}
{'end': '2022-06-13T00:00:00.000Z', 'start': '2022-06-12T00:00:00.000Z', 'tweet_count': 1827583}
{'end': '2022-06-13T17:21:19.000Z', 'start': '2022-06-13T00:00:00.000Z', 'tweet_count': 1279660}


In [31]:
def create_tweet(text):
    client = tweepy.Client(consumer_key=api_key,
        consumer_secret=api_secret_key,
        access_token=access_token,
        access_token_secret=access_secret_token)
    response=client.create_tweet(text=text)

#create_tweet("Hello World!")

In [39]:
def determine_file_name(query):
    file_name = "Data/"+query+".csv"
    return file_name

In [40]:
def search_tweets(query):
    client = tweepy.Client(bearer_token=bearer_token)
    response = client.search_recent_tweets(query=query)
    return response

query = "NFT"

response = search_tweets(query)
#for tweet in response.data:
   # print(tweet.text)

In [114]:
#this function writes the event data to a csv file to avoid calling the API multiple times
def write_csv(data_list, file_name):
    with open(file_name, mode='w', encoding='utf-8', newline='\n') as f:
        header = "Tweets"
        f.write("%s\n" % header)
        for data in data_list:
            f.write("%s\n" % data)

In [117]:
def sanitize_tweet(tweet):
    sanitzed_tweet = tweet.replace("\n", " ")
    sanitzed_tweet = re.sub(r'@[A-za-z0-9]+','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'#','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'RT[\s]','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'https?:\/\/\S+','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r',+','',sanitzed_tweet)
    return sanitzed_tweet

In [129]:
#get the max data
limit = 500

max_results = 100
tweet_list=[]
def search_tweets1(query,reload=False):
    client = tweepy.Client(bearer_token=bearer_token)
    file_name = determine_file_name(query)
    path = Path(file_name)
    
    i=1
    if (reload == False and path.is_file()):
        print(f'The file {file_name} exists')
        nft_df = pd.read_csv(file_name)
        return nft_df
    else:
        for tweet in tweepy.Paginator(client.search_recent_tweets,query=query,max_results=max_results).flatten(limit=limit):
            #print(str(i) +")" + tweet.text)
            i=i+1
            sanitized_tweet = sanitize_tweet(tweet.text)
            tweet_list.append(sanitized_tweet)
        #once the list is generated, write it to file
        write_csv(tweet_list,file_name)
    #return list as a dataframe
    return pd.DataFrame(tweet_list)

query = "NFT"
is_reload_needed = False
tweet_df = search_tweets1(query,is_reload_needed)
tweet_df


,0
0,: SHIBURAI and VIRALCRYPTO are prime examples ...
1,: 0.4 ETH ($1000) NFT Giveaway 🥽🥽 To enter ...
2,Free NFT
3,: Will we see the bulls regain control for a g...
4,: 🔥 BIG WL GIVEAWAY 🔥 2x WL for each Collecti...
...,...
495,: Meme contest in our discord tomorow. 🎁 Join ...
496,: Yoooooo. Some people are doom and gloom righ...
497,Ron blames Santa Claus for him becoming a g...
498,: $200 • 24 hrs • rt • follow me ➡️ follow ...
